# Term Frequency–Inverse Document Frequency (TF-IDF) - Experimento

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declare parâmetros e hiperparâmetros para o modelo
Os componentes podem declarar (e usar) estes parâmetros como padrão:
- dataset
- target

Use estes parâmetros para carregar/salvar conjutos de dados, modelos, métricas e figuras com a ajuda do [SDK da PlatIAgro](https://platiagro.github.io/sdk/). <br>
É possível também declarar parâmetros personalizados para serem definidos ao executar um experimento. 

Selecione os hiperparâmetros e seus respectivos valores para serem usados ao treinar o modelo:
- language

Estes parâmetros são alguns dos oferecidos pela classe do modelo, você também pode utilizar outros existentes. <br>
Dê uma olhada nos [parâmetros do modelo](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer) para mais informações.

In [ ]:
# parâmetros
dataset = "imdb" #@param {type:"string"}
target = "label" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
text = "text" #@param {type:"string", label:"Texto alvo", description:"Nome da coluna do texto alvo pertencente ao dataset."}
language = "english" #@param ["portuguese", "english"] {type:"string", label:"Linguagem", description:"Linguagem da qual os stopwords pertencem. Deve ser a mesma utilizada no dataset."}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df[text].to_numpy()
y = df[target].to_numpy()

## Acesso ao conjunto de dados

Utiliza a função load_dataset do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Divisão do datset em subconjuntos de treino e teste

Subconjunto de Treino: amostras de dados usado para treinar o modelo (``fit``). <br>
Subconjunto de Teste: a amostra de dados usada para fornecer uma avaliação imparcial de um modelo adequado ao conjunto de dados de treinamento.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

## Busca por stopwords

Stopwords (ou palavras de parada) são palavras que geralmente se referem às mais comuns em um idioma ou em um corpus. <br>
Elas podem ser ignoradas com segurança sem sacrificar o significado da frase, pois são palarvas que não agregram muito significado.

In [ ]:
import nltk

# Download stopwords from nltk
nltk.download('stopwords')

# Get a list of stopwords for the defined language
stopwords = nltk.corpus.stopwords.words(language)

## Processamento do texto

Funções auxiliares para processamento dos dados.

In [ ]:
from re import sub
from collections import defaultdict
from functools import reduce


def tokenize_without_punctuation(text_list: list = None):
    """Tokenize without ponctuation.

    Args:
        text_list (list): a list of texts to be used.

    Returns:
        A list of tokenized text without punctuation.
    """
    tokenize_list = list()
    punctuation_pattern = "[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]"
    html_tag_pattern = "<.*?>"

    for text in text_list:
        text = sub(html_tag_pattern, ' ', text)
        tokenize_list.append(sub(punctuation_pattern, ' ', text).split(' '))

    return tokenize_list


def top_tokens_stopwords(token_list: list, percentage: float = 0.01):
    """Selects the most relevant stops words of the tokerized texts.

    Args:
        token_list (list): list of tokens.
        percentage (float): percentage threshold.
    """
    vocabulary = defaultdict(int)

    for sample in token_list:
        for token in sample:
            vocabulary[token] += 1

    all_tokens = sorted(vocabulary.items(), key=lambda token: token[1], reverse=True)
    top_tokens = all_tokens[:int(len(all_tokens) * percentage)]

    return [token[0] for token in top_tokens]


def remove_specific_tokens(token_list: list, tokens_to_be_removed: list = None):
    """Removes specific tokens from a token list.

    Args:
        token_list (list): list of tokens from which other tokens will be removed.
        tokens_to_be_removed (list): list of tokens that need to be removed.
    """
    token_list_ = list()

    if tokens_to_be_removed is None:
        tokens_to_be_removed = top_tokens_stopwords(token_list)

    for sample in token_list:
        sample = list(set(sample) - set(tokens_to_be_removed))
        token_list_.append(sample)

    return token_list_


def token_restructuring(token_list: list):
    """Reduce a nested list of tokens to a single list (1D).
    
    Args:
        token_list (list): list to be work on.
    """
    return reduce(lambda x, y: x + y, token_list)

## Treina modelo usando sklearn.feature_extraction.text.TfidfVectorizer

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

tokenize_without_punctuation_ = FunctionTransformer(tokenize_without_punctuation, validate=False)
remove_specific_tokens_ = FunctionTransformer(remove_specific_tokens, validate=False)
token_restructuring_ = FunctionTransformer(token_restructuring, validate=False)

pipeline = Pipeline(steps=[
    ('remove_punctuation', tokenize_without_punctuation_),
    ('remove_top_tokens', remove_specific_tokens_),
    ('restructuring', token_restructuring_),
    ('estimator', TfidfVectorizer())
])

tdidf_matrix = pipeline.fit_transform(X_train, y_train)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=pd.DataFrame(tdidf_matrix.toarray()))

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline)